In [113]:
import sqlite3
import pandas as pd
import time
import os
import warnings
import plotly.express as px
import plotly.io as pio
pd.set_option('display.width', 500)

In [114]:
inat_con = sqlite3.connect("inaturalist-open-data-20220127.sq3db")
print(inat_con.total_changes)

0


In [ ]:
# insect_rank_df = pd.read_csv('iNaturalist_Insects_Ranks.csv')
# print(insect_rank_df.head())

In [115]:
# insect_con = sqlite3.connect("ranked-insects-inaturalist-open-data-20220127.sq3db")
# print(insect_con.total_changes)

0


In [116]:
# start_time = time.time()
# insect_rank_count_df = pd.read_sql_query(
#     """
#     SELECT COUNT(photo_id) as photo_count, taxon_name, ancestry, rank, rank_level   
#     FROM insects 
#     GROUP BY taxon_id 
#     ORDER BY COUNT(photo_id) DESC 
#     """,
#     insect_con
# )

# print(insect_rank_count_df)
#insect_rank_count_df.to_csv('iNaturalist_Insects_Rank_Count.csv', index=False)
# print("Executed in %.2f seconds" % (time.time() - start_time))

        photo_count                 taxon_name                                           ancestry         rank  rank_level
0            190328             Apis mellifera  48460/1/47120/372739/47158/184884/47201/124417...      species        10.0
1            142726           Danaus plexippus  48460/1/47120/372739/47158/184884/47157/47224/...      species        10.0
2            119322          Harmonia axyridis  48460/1/47120/372739/47158/184884/47208/71130/...      species        10.0
3             78427           Bombus impatiens  48460/1/47120/372739/47158/184884/47201/124417...      species        10.0
4             58325  Coccinella septempunctata  48460/1/47120/372739/47158/184884/47208/71130/...      species        10.0
...             ...                        ...                                                ...          ...         ...
107969            1                     Neduba  48460/1/47120/372739/47158/184884/47651/132694...        genus        20.0
107970          

In [117]:
insect_rank_count_df = pd.read_csv('iNaturalist_Insects_Rank_Count.csv')
print(insect_rank_df.head())

In [ ]:
sample_insect_rank_df = insect_rank_count_df[0:5000]
print(sample_insect_rank_df)

In [88]:
start_time = time.time()
taxa_df = pd.read_sql(
    """
    SELECT name, taxon_id, ancestry, rank, rank_level 
    FROM taxa 
    """,
    inat_con
)
print(taxa_df.head())
print("Executed in %.2f seconds" % (time.time() - start_time))

                       name  taxon_id                            ancestry     rank  rank_level
0                      name  taxon_id                            ancestry     rank  rank_level
1         Icterus parisorum      9318  48460/1/2/355675/3/7251/11989/9315  species        10.0
2                 Calyptura      8943      48460/1/2/355675/3/7251/980017    genus        20.0
3  Ictinia mississippiensis      5416  48460/1/2/355675/3/71261/5067/5414  species        10.0
4   Amazonetta brasiliensis      7182   48460/1/2/355675/3/6888/6912/7181  species        10.0
Executed in 6.72 seconds


In [110]:
def make_taxa_column(row, taxa_df, rank):
    ancestry_list = row.ancestry.split('/')
    for i in range(0, len(ancestry_list)):
        taxa = taxa_df.loc[taxa_df['taxon_id'] == int(ancestry_list[i])]
        if (taxa['rank'].values == rank):
            return taxa.name
        else:
            continue
    return 'No ' + rank

def make_species(row, taxa_df, sample_df, rank):
    if str(row['rank']) == rank:
        return row.taxon_name
    else:
        index = sample_df.index
        condition = sample_df['taxon_name'] == row.taxon_name
        indices = index[condition]
        indices_list = indices.tolist()
        
        #indicies_list = sample_df.index[sample_df['taxon_name'] == row.taxon_name].tolist()
        return  str(indices_list) + ' No ' + rank
    
def make_subspecies(row, taxa_df, rank):
    if str(row['rank']) == rank:
        return row.taxon_name
    else:
        return None
    

In [ ]:
# #4778 seconds
# start_time = time.time()
# try:
#     sample_insect_rank_df['state_of_matter'] = sample_insect_rank_df.apply(make_taxa_column, taxa_df=taxa_df, rank='stateofmatter', axis=1).bfill(axis=1).iloc[:, 0]
# except:
#     print("No state_of_matter")
# try:
#     sample_insect_rank_df['kingdom'] = sample_insect_rank_df.apply(make_taxa_column, taxa_df=taxa_df, rank='kingdom', axis=1).bfill(axis=1).iloc[:, 0]
# except:
#     print("No kingdom")
# try:
#     sample_insect_rank_df['phylum'] = sample_insect_rank_df.apply(make_taxa_column, taxa_df=taxa_df, rank='phylum', axis=1).bfill(axis=1).iloc[:, 0]
# except:
#     print("No phylum")
# try:
#     sample_insect_rank_df['subphylum'] = sample_insect_rank_df.apply(make_taxa_column, taxa_df=taxa_df, rank='subphylum', axis=1).bfill(axis=1).iloc[:, 0]
# except:
#     print("No subphylum")
# try:
#     sample_insect_rank_df['class'] = sample_insect_rank_df.apply(make_taxa_column, taxa_df=taxa_df, rank='class', axis=1).bfill(axis=1).iloc[:, 0]
# except:
#     print("No class")
# try:
#     sample_insect_rank_df['subclass'] = sample_insect_rank_df.apply(make_taxa_column, taxa_df=taxa_df, rank='subclass', axis=1).bfill(axis=1).iloc[:, 0]
# except:
#     print("No subclass")
# try:
#     sample_insect_rank_df['order'] = sample_insect_rank_df.apply(make_taxa_column, taxa_df=taxa_df, rank='order', axis=1).bfill(axis=1).iloc[:, 0]
# except:
#     print("No order")
# try:
#     sample_insect_rank_df['superfamily'] = sample_insect_rank_df.apply(make_taxa_column, taxa_df=taxa_df, rank='superfamily', axis=1).bfill(axis=1).iloc[:, 0]
# except:
#     print("No superfamily")
# try:
#     sample_insect_rank_df['family'] = sample_insect_rank_df.apply(make_taxa_column, taxa_df=taxa_df, rank='family', axis=1).bfill(axis=1).iloc[:, 0]
# except:
#     print("No family")
# try:
#     sample_insect_rank_df['subfamily'] = sample_insect_rank_df.apply(make_taxa_column, taxa_df=taxa_df, rank='subfamily', axis=1).bfill(axis=1).iloc[:, 0]
# except:
#     print("No subfamily")
# try:
#     sample_insect_rank_df['supertribe'] = sample_insect_rank_df.apply(make_taxa_column, taxa_df=taxa_df, rank='supertribe', axis=1).bfill(axis=1).iloc[:, 0]
# except:
#     print("No supertribe")
# try:
#     sample_insect_rank_df['tribe'] = sample_insect_rank_df.apply(make_taxa_column, taxa_df=taxa_df, rank='tribe', axis=1).bfill(axis=1).iloc[:, 0]
# except:
#     print("No tribe")
# try:
#     sample_insect_rank_df['subtribe'] = sample_insect_rank_df.apply(make_taxa_column, taxa_df=taxa_df, rank='subtribe', axis=1).bfill(axis=1).iloc[:, 0]
# except:
#     print("No subtribe")
# try:
#     sample_insect_rank_df['genus'] = sample_insect_rank_df.apply(make_taxa_column, taxa_df=taxa_df, rank='genus', axis=1).bfill(axis=1).iloc[:, 0]
# except:
#     print("No genus")
# try:
#     sample_insect_rank_df['subgenus'] = sample_insect_rank_df.apply(make_taxa_column, taxa_df=taxa_df, rank='subgenus', axis=1).bfill(axis=1).iloc[:, 0]
# except:
#     print("No subgenus")
# try:
#     sample_insect_rank_df['section'] = sample_insect_rank_df.apply(make_taxa_column, taxa_df=taxa_df, rank='section', axis=1).bfill(axis=1).iloc[:, 0]
# except:
#     print("No section")
# try:
#     sample_insect_rank_df['complex'] = sample_insect_rank_df.apply(make_taxa_column, taxa_df=taxa_df, rank='complex', axis=1).bfill(axis=1).iloc[:, 0]
# except:
#     print("No complex")
# try:
#     sample_insect_rank_df['species'] = sample_insect_rank_df.apply(make_taxa_column, taxa_df=taxa_df, rank='species', axis=1).bfill(axis=1).iloc[:, 0]
# except:
#     print("No species")
# # try:
# #     sample_insect_rank_df['subspecies'] = sample_insect_rank_df.apply(make_taxa_column, taxa_df=taxa_df, rank='subspecies', axis=1).bfill(axis=1).iloc[:, 0]
# # except:
# #     print("No subspecies")
# sample_insect_rank_df['species'] = sample_insect_rank_df.apply(make_species, taxa_df=taxa_df, sample_df = sample_insect_rank_df, rank='species', axis=1)
# sample_insect_rank_df['subspecies'] = sample_insect_rank_df.apply(make_subspecies, taxa_df=taxa_df, rank='subspecies', axis=1)
# print(sample_insect_rank_df.head())
# sample_insect_rank_df.to_csv('iNaturalist_Insects_Taxa_Columns_T5000.csv', index=False)
# print("Executed in %.2f seconds" % (time.time() - start_time))

In [120]:
sample_insect_rank_df = pd.read_csv('iNaturalist_Insects_Taxa_Columns_T5000.csv')
print(sample_insect_rank_df.head())

   photo_count                 taxon_name                                           ancestry     rank  rank_level state_of_matter   kingdom      phylum subphylum    class   subclass        order     superfamily         family      subfamily         tribe       genus     subgenus
0       190328             Apis mellifera  48460/1/47120/372739/47158/184884/47201/124417...  species        10.0            Life  Animalia  Arthropoda  Hexapoda  Insecta  Pterygota  Hymenoptera         Apoidea         Apidae         Apinae         Apini        Apis         Apis
1       142726           Danaus plexippus  48460/1/47120/372739/47158/184884/47157/47224/...  species        10.0            Life  Animalia  Arthropoda  Hexapoda  Insecta  Pterygota  Lepidoptera   Papilionoidea    Nymphalidae       Danainae       Danaini      Danaus  No subgenus
2       119322          Harmonia axyridis  48460/1/47120/372739/47158/184884/47208/71130/...  species        10.0            Life  Animalia  Arthropoda  Hexapod

In [124]:
warnings.filterwarnings('ignore')
pio.renderers.default = "browser"
#fig = px.sunburst(sample_insect_rank_df, path=['state_of_matter', 'kingdom', 'phylum', 'subphylum', 'class', 'subclass', 'order', 'superfamily', 'family', 'subfamily', 'tribe', 'genus', 'subgenus', 'taxon_name'], values='photo_count', color='photo_count')
fig = px.sunburst(sample_insect_rank_df, path=['class', 'subclass', 'order', 'superfamily', 'family', 'subfamily', 'tribe', 'genus', 'subgenus', 'species', 'subspecies'], values='photo_count', color='photo_count')
fig.show()